In [ ]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

import pandas as pd
from openpyxl import load_workbook

import configparser 
config = configparser.ConfigParser()
config.read('config.ini')
ip = config['DEFAULT']['IP-LOCAL']
port = config['DEFAULT']['MongoDB-Port']

from pymongo import MongoClient
client = MongoClient(ip, int(port))

import twitter

consumer_key = config['TWITTER']['CK']
consumer_secret = config['TWITTER']['CS']
access_token_key = config['TWITTER']['ATK']
access_token_secret = config['TWITTER']['ATS']

api = twitter.Api(consumer_key=consumer_key,
                  consumer_secret=consumer_secret,
                  access_token_key=access_token_key,
                  access_token_secret=access_token_secret,
                  tweet_mode= 'extended',
                  sleep_on_rate_limit=True)

In [ ]:
db_twitter = client["Twitter"]

In [ ]:
number_truncated_tweets=db_twitter["twitter-small"].find({"truncated":True}).count()
print("number_truncated_tweets:", number_truncated_tweets)

In [ ]:
# iterate the cursor
for document in db_twitter["twitter-small"].find({"truncated":True}):
    try:
        full_text = api.GetStatus(document["id_str"]).full_text
        db_twitter["twitter-small"].update({"_id": document["_id"]}, {"$set": {"full_text": full_text,"truncated":False}})
    except:
        pass